# Avoiding Data Gaps with Strategic Forecasting

One often encounters the challenge of forecasting over a period where no immediate past data is available. Imagine for example making predictions for a future period, such as the entire months of November and December, when your latest available data concludes at the end of October.

Two forecasting strategies can be used to tackle this challenge:

+ **Extensive Recursive Forecasting**: This method involves making predictions over the entire target period (in our case, two months). While straightforward, this approach has inherent limitations and risks such as error accumulation. In recursive forecasting, since each prediction is used as an input for subsequent forecasts, this can lead to an accumulation of errors over time, especially for longer forecast horizons.

<p style="text-align: center">
<img src="../img/extensive_recursive_forecasting.gif" style="width: 650px;">
<br>
<font size="2.5"> <i>Extensive Recursive Forecasting.</i></font>
</p>

+ **Training with Extended Lag Periods**: An alternative and often more robust strategy involves training the forecasting model using lags from a period significantly earlier than the gap. This technique allows for updating the model with the most recent data points before making new predictions, ensuring more accuracy and adaptability.

<p style="text-align: center">
<img src="../img/predict_extended_lags_periods.gif" style="width: 650px;">
<br>
<font size="2.5"> <i>Predictions with extended lags periods.</i></font>
</p>

<div class="admonition note" name="html-admonition" style="background: rgba(0,184,212,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00b8d4; border-color: #00b8d4; padding-left: 10px;">

<p class="title">
    <i style="font-size: 18px; color:#00b8d4;"></i>
    <b style="color: #00b8d4;">&#9998 Note</b>
</p>

Link a user guide forecaster in production?

</div>

## Libraries

In [98]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster

## Data

In [99]:
# Download data and preprocessing
# ==============================================================================
data = fetch_dataset(name='h2o', raw=True, verbose=False,
                     kwargs_read_csv={'header': 0, 'names': ['y', 'datetime']})

data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data.sort_index()

data.head(3)

,y
datetime,
1991-07-01,0.429795
1991-08-01,0.400906
1991-09-01,0.432159


Train dates      : 1991-07-01 00:00:00 --- 2005-06-01 00:00:00  (n=168)
Validation dates : 2005-07-01 00:00:00 --- 2008-06-01 00:00:00  (n=36)


In [100]:
# Train-validation dates
# ==============================================================================
end_train = '2005-06-01 23:59:59'
data_train = data[:end_train].copy()
data_test  = data[end_train:].copy()

print(
    f"Train dates      : {data.index.min()} --- {data.loc[:end_train].index.max()}"
    f"  (n={len(data.loc[:end_train])})"
)
print(
    f"Validation dates : {data.loc[end_train:].index.min()} --- {data.index.max()}"
    f"  (n={len(data.loc[end_train:])})"
)

# Plot
# ==============================================================================
fig, ax = plt.subplots(figsize=(6, 3))
data.loc[:end_train].plot(ax=ax, label='train')
data.loc[end_train:].plot(ax=ax, label='validation')
ax.legend()
plt.show()

## Extensive Recursive Forecasting

In [101]:
# Create and fit forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = Ridge(random_state=123),
                 lags      = 11
             )

forecaster.fit(y=data_train['y'])

In [102]:
# Predict whole test data
# ==============================================================================
predictions = forecaster.predict(steps=len(data_test))
predictions.head(3)

2005-07-01    0.919256
2005-08-01    1.011133
2005-09-01    1.119494
Freq: MS, Name: pred, dtype: float64

In [103]:
# Plot predictions
# ==============================================================================
fig, ax = plt.subplots(figsize=(6, 3))
data_train.plot(ax=ax, label='train')
data_test.plot(ax=ax, label='test')
predictions.plot(ax=ax, label='predictions')
ax.legend();

In [104]:
# Prediction error
# ==============================================================================
error_mse = mean_absolute_error(
                y_true = data_test,
                y_pred = predictions
            )

print(f"Test error (MAE): {error_mse}")

Test error (MAE): 0.09633923180522949


## Training with Extended Lag Periods

In [105]:
len(data_test)

36

In [106]:
# Create and fit forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = Ridge(random_state=123),
                 lags      = range(36, 48) 
             )

forecaster.fit(y=data_train['y'])

<div class="admonition note" name="html-admonition" style="background: rgba(0,191,191,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00bfa5; border-color: #00bfa5; padding-left: 10px; padding-right: 10px;">

<p class="title">
    <i style="font-size: 18px; color:#00bfa5;"></i>
    <b style="color: #00bfa5;">&#128161 Tip</b>
</p>

To a better understanding of backtesting process visit the <a href="https://skforecast.org/latest/user_guides/backtesting">Backtesting user guide</a>

</div>

In [107]:
# Backtesting forecaster
# ==============================================================================
metric, predictions = backtesting_forecaster(
                          forecaster            = forecaster,
                          y                     = data['y'],
                          steps                 = 2,
                          metric                = 'mean_absolute_error',
                          initial_train_size    = len(data_train),
                          refit                 = False,
                          n_jobs                = 'auto',
                          verbose               = False,
                          show_progress         = True  
                      )

print(f"Backtest error: {metric}")

  0%|          | 0/18 [00:00<?, ?it/s]

Backtest error: 0.08592790691982011


In [108]:
# Plot predictions
# ==============================================================================
fig, ax = plt.subplots(figsize=(6, 3))
data_train.plot(ax=ax, label='train')
data_test.plot(ax=ax, label='test')
predictions.plot(ax=ax, label='predictions')
ax.legend();

## Predictions with Extended Lag Periods